In [1]:
import pandas as pd
import sys
import os
import numpy as np
from collections import Counter

In [2]:
#load deezer test and train datasets, combine
df1 = pd.read_csv("data/deezer_train.csv")
df2 = pd.read_csv("data/deezer_test.csv")
deezer_df = pd.concat([df1,df2])
deezer_df

,dzr_sng_id,MSD_sng_id,MSD_track_id,valence,arousal,artist_name,track_name
0,213754,SORORTY12A67ADD67E,TRJIEJX12903CF2202,-0.682725,0.316758,Dark Tranquillity,Insanity's Crescendo
1,216752,SOVJXGK12A8AE4760C,TRLBHVY128F4242C31,-1.400403,1.538229,Anorexia Nervosa,Stabat mater dolorosa
2,239138,SOACXLC12AB018218D,TRQJZHB128F92FA42D,-1.935250,-0.655810,Jimmy Witherspoon,Ain't Nobody's Business
3,254391,SOUTMGD12A8AE47448,TRCXLKK128F42366BF,0.815393,0.662457,Uniting Nations,Out Of Touch
4,266313,SOTITAA12A6D4F8DA4,TRPMWQM128F932C0C8,1.257460,1.086515,The Walkmen,Brandy alexander
...,...,...,...,...,...,...,...
3509,113398772,SOICSEW12AC4688D1D,TRKIVZW12903CF273F,-1.700572,1.842444,Sworn Enemy,We Hate
3510,114388304,SOACEJK12A8C136E19,TRUOTTN128F425C099,1.178325,1.183311,Van Morrison,Wild Night
3511,114388306,SOTWLJL12A58A7DA90,TRQOBCM128F428EA6E,0.373325,-0.923151,Van Morrison,Tupelo Honey
3512,114388348,SOVDDUT12A8C13B81D,TRHSGAZ128F428EA74,0.815393,0.662457,Van Morrison,Once In A Blue Moon


In [3]:
#separate mxm bag of words into a list
txt = open("data/mxm_words_train.txt","r")
words = txt.read().split(',')
print(words)


['i', 'the', 'you', 'to', 'and', 'a', 'me', 'it', 'not', 'in', 'my', 'is', 'of', 'your', 'that', 'do', 'on', 'are', 'we', 'am', 'will', 'all', 'for', 'no', 'be', 'have', 'love', 'so', 'know', 'this', 'but', 'with', 'what', 'just', 'when', 'like', 'now', 'que', 'time', 'can', 'come', 'de', 'there', 'go', 'up', 'oh', 'la', 'one', 'they', 'out', 'down', 'get', 'she', 'was', 'see', 'if', 'got', 'never', 'from', 'he', 'feel', 'want', 'let', 'make', 'way', 'say', 'take', 'would', 'as', 'ca', 'day', 'at', 'baby', 'away', 'life', 'yeah', 'y', 'back', 'by', 'her', 'heart', 'here', 'how', 'could', 'night', 'need', 'our', 'look', 'where', 'en', 'eye', 'thing', 'world', 'more', 'cause', 'gonna', 'die', 'right', 'been', 'tell', 'think', 'un', 'who', 'el', 'through', 'man', 'live', 'again', 'give', 'too', 'only', 'te', 'try', 'tu', 'or', 'why', 'se', 'keep', 'dream', 'well', 'mind', 'an', 'wo', 'still', 'us', 'his', 'long', 'mi', 'girl', 'wanna', 'find', 'fall', 'around', 'good', 'about', 'some', 'o

In [4]:
#load mxm test and train sets, combine, and label columns
df3 = pd.read_csv("data/mxm_train.csv", names=list(range(730)), low_memory=False)
df4 = pd.read_csv("data/mxm_test.csv", names=list(range(730)), low_memory=False)
mxm_df = pd.concat([df3,df4])
col = ["MSD_track_id","mxm_id"]
for i in range (0,728):
    stri = str(i)
    col.append(stri)
mxm_df.columns = col
mxm_df

,MSD_track_id,mxm_id,0,1,2,3,4,5,6,7,...,718,719,720,721,722,723,724,725,726,727
0,MSD_track_id,mxm_track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAAAAV128F421A322,4623710,1:6,2:4,3:2,4:2,5:5,6:3,7:1,8:1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAAABD128F429CF47,6477168,1:10,3:17,4:8,5:2,6:2,7:1,8:3,9:2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAAAED128E0783FAB,2516445,1:28,2:15,3:2,4:12,5:22,6:2,7:2,8:4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRAAAEF128F4273421,3759847,1:5,2:4,3:3,4:2,5:1,6:11,9:4,12:9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27139,TRZZXFY128F9342D0E,1265451,1:24,2:3,3:22,4:7,5:4,6:15,7:12,8:5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27140,TRZZXOQ128F932A083,4292070,1:12,4:2,5:1,8:1,9:3,11:1,13:2,20:2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27141,TRZZXVN128F93285B4,7528751,10:7,22:1,28:4,54:2,97:7,102:2,106:1,122:2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27142,TRZZYLF128F9316CAB,3748433,10:5,28:2,41:3,47:12,102:4,112:6,114:15,117:4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#merge deezer and mxm datasets, drop unused columns
df = pd.merge(mxm_df,deezer_df,on="MSD_track_id")
df = df.drop(["mxm_id","dzr_sng_id","MSD_sng_id"],axis=1)
df


,MSD_track_id,0,1,2,3,4,5,6,7,8,...,722,723,724,725,726,727,valence,arousal,artist_name,track_name
0,TRAABJS128F9325C99,1:5,2:3,3:3,4:3,5:8,7:1,15:1,21:3,23:2,...,NaN,NaN,NaN,NaN,NaN,NaN,0.766274,-2.112357,Beach House,Auburn and Ivory
1,TRAADQX128F422B4CF,1:8,2:4,3:10,4:4,6:1,7:1,8:5,9:2,11:2,...,NaN,NaN,NaN,NaN,NaN,NaN,0.957291,0.390507,Casiotone For The Painfully Alone,Nashville Parthenon
2,TRAADVO128E07999E9,1:2,2:9,3:8,4:7,5:2,6:1,8:7,9:3,12:4,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.162886,0.148517,Jamie Cullum,Oh God
3,TRAAFGQ128F427D884,1:24,2:11,3:5,4:13,5:19,6:16,7:9,8:7,9:5,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.019460,-0.196721,The Kooks,One Last Time
4,TRAAGEC128E079252C,1:13,2:8,3:8,4:4,5:4,7:3,8:1,9:1,10:2,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.767318,-0.190268,Nine Inch Nails,Metal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10711,TRZWCWI128F429753C,1:53,2:16,3:33,4:12,5:5,6:8,7:4,8:2,9:2,...,NaN,NaN,NaN,NaN,NaN,NaN,1.541257,1.704165,Guy Sebastian,All I Need Is You
10712,TRZWXMQ12903CE6AEF,1:18,2:17,3:18,4:10,5:13,6:15,7:2,8:9,9:10,...,NaN,NaN,NaN,NaN,NaN,NaN,0.373325,-0.923151,Jurassic 5 / Nelly Furtado,Thin Line
10713,TRZYAGJ128F9332CEF,1:20,2:20,3:36,4:2,5:5,6:2,7:8,8:13,9:7,...,NaN,NaN,NaN,NaN,NaN,NaN,1.541257,1.704165,Justin Bieber,Favorite Girl
10714,TRZYPOT128F14B0B7C,1:5,2:4,3:3,4:1,5:5,6:8,8:3,10:7,11:1,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.317974,-0.399224,Okkervil River,In A Radio Song


In [6]:
#normalize valence and arousal scores to 0 (low) or 1 (high)
v_norm = []
for i in df["valence"]:
    if i > 0.4:
        v_norm.append(1)
    elif i < -0.4:
        v_norm.append(0)
    else:
        v_norm.append(0.5)

a_norm = []
for i in df["arousal"]:
    if i > 0:
        a_norm.append(1)
    elif i < 0:
        a_norm.append(0)
df['v_norm'] = v_norm
df['a_norm'] = a_norm


In [7]:
#compile song lyrics into a bag-of-words list
lyrics_col = []
for index, row in df.iterrows():
    lyrics = []
    for i in range(0,728):
        stri = str(i)
        if isinstance(row[stri],str):
            line = row[stri].split(':')
            for j in range (int(line[1])):
                lyrics.append(words[int(line[0])-1])
    lyrics_col.append(lyrics)

df['lyrics'] = lyrics_col

In [8]:
#drop mxm song index columns
df = df.drop(columns=df.loc[:, '0':'727'].columns)
df

,MSD_track_id,valence,arousal,artist_name,track_name,v_norm,a_norm,lyrics
0,TRAABJS128F9325C99,0.766274,-2.112357,Beach House,Auburn and Ivory,1.0,0,"[i, i, i, i, i, the, the, the, you, you, you, ..."
1,TRAADQX128F422B4CF,0.957291,0.390507,Casiotone For The Painfully Alone,Nashville Parthenon,1.0,1,"[i, i, i, i, i, i, i, i, the, the, the, the, y..."
2,TRAADVO128E07999E9,-0.162886,0.148517,Jamie Cullum,Oh God,0.5,1,"[i, i, the, the, the, the, the, the, the, the,..."
3,TRAAFGQ128F427D884,-1.019460,-0.196721,The Kooks,One Last Time,0.0,0,"[i, i, i, i, i, i, i, i, i, i, i, i, i, i, i, ..."
4,TRAAGEC128E079252C,-0.767318,-0.190268,Nine Inch Nails,Metal,0.0,0,"[i, i, i, i, i, i, i, i, i, i, i, i, i, the, t..."
...,...,...,...,...,...,...,...,...
10711,TRZWCWI128F429753C,1.541257,1.704165,Guy Sebastian,All I Need Is You,1.0,1,"[i, i, i, i, i, i, i, i, i, i, i, i, i, i, i, ..."
10712,TRZWXMQ12903CE6AEF,0.373325,-0.923151,Jurassic 5 / Nelly Furtado,Thin Line,0.5,0,"[i, i, i, i, i, i, i, i, i, i, i, i, i, i, i, ..."
10713,TRZYAGJ128F9332CEF,1.541257,1.704165,Justin Bieber,Favorite Girl,1.0,1,"[i, i, i, i, i, i, i, i, i, i, i, i, i, i, i, ..."
10714,TRZYPOT128F14B0B7C,-0.317974,-0.399224,Okkervil River,In A Radio Song,0.5,0,"[i, i, i, i, i, the, the, the, the, you, you, ..."


In [27]:
train_df = df.iloc[:1000,:]
test_df = df.iloc[1000:,:]

In [10]:
train_df

,MSD_track_id,valence,arousal,artist_name,track_name,v_norm,a_norm,lyrics
0,TRAABJS128F9325C99,0.766274,-2.112357,Beach House,Auburn and Ivory,1.0,0,"[i, i, i, i, i, the, the, the, you, you, you, ..."
1,TRAADQX128F422B4CF,0.957291,0.390507,Casiotone For The Painfully Alone,Nashville Parthenon,1.0,1,"[i, i, i, i, i, i, i, i, the, the, the, the, y..."
2,TRAADVO128E07999E9,-0.162886,0.148517,Jamie Cullum,Oh God,0.5,1,"[i, i, the, the, the, the, the, the, the, the,..."
3,TRAAFGQ128F427D884,-1.019460,-0.196721,The Kooks,One Last Time,0.0,0,"[i, i, i, i, i, i, i, i, i, i, i, i, i, i, i, ..."
4,TRAAGEC128E079252C,-0.767318,-0.190268,Nine Inch Nails,Metal,0.0,0,"[i, i, i, i, i, i, i, i, i, i, i, i, i, the, t..."
...,...,...,...,...,...,...,...,...
995,TRCJTWS128F4280BB1,0.116817,-0.658114,John Mayer,Quiet,0.5,0,"[i, i, i, the, the, the, the, the, the, the, t..."
996,TRCJUCJ128F4276F88,1.262918,0.846830,Angels and Airwaves,Heaven,1.0,1,"[i, i, i, i, i, i, i, i, i, i, i, i, i, i, the..."
997,TRCJUCQ12903CEB017,1.178325,1.183311,OutKast,Hey Ya!,1.0,1,"[i, i, i, i, i, the, the, the, the, the, the, ..."
998,TRCJUYN128F92E3CE3,-0.188810,-0.798699,SONATA ARCTICA,Last Drop Falls,0.5,0,"[i, i, i, i, i, i, i, i, i, i, i, i, i, i, i, ..."


In [11]:
test_df

,MSD_track_id,valence,arousal,artist_name,track_name,v_norm,a_norm,lyrics
1000,TRCJYPQ128F4282943,-1.700572,1.842444,Nellie McKay,Inner Peace,0.0,1,"[i, i, i, i, i, i, i, i, i, i, i, i, i, i, i, ..."
1001,TRCJZEY128F9337970,-1.333820,-0.030785,Marilyn Manson,Just A Car Crash Away,0.0,0,"[i, i, i, i, i, i, i, i, the, you, you, you, y..."
1002,TRCJZXG128F427E24E,0.373325,-0.923151,U2,Until The End Of The World,0.5,0,"[i, i, i, i, i, i, i, i, i, the, the, the, the..."
1003,TRCKARW128F42659CF,1.541257,1.704165,Andrew W.K.,I Get Wet,1.0,1,"[i, i, i, i, i, i, i, i, i, i, i, i, i, i, i, ..."
1004,TRCKBXY128E078B448,1.541257,1.704165,Annie,No Easy Love,1.0,1,"[i, i, i, i, i, i, i, i, i, i, i, i, i, i, i, ..."
...,...,...,...,...,...,...,...,...
10711,TRZWCWI128F429753C,1.541257,1.704165,Guy Sebastian,All I Need Is You,1.0,1,"[i, i, i, i, i, i, i, i, i, i, i, i, i, i, i, ..."
10712,TRZWXMQ12903CE6AEF,0.373325,-0.923151,Jurassic 5 / Nelly Furtado,Thin Line,0.5,0,"[i, i, i, i, i, i, i, i, i, i, i, i, i, i, i, ..."
10713,TRZYAGJ128F9332CEF,1.541257,1.704165,Justin Bieber,Favorite Girl,1.0,1,"[i, i, i, i, i, i, i, i, i, i, i, i, i, i, i, ..."
10714,TRZYPOT128F14B0B7C,-0.317974,-0.399224,Okkervil River,In A Radio Song,0.5,0,"[i, i, i, i, i, the, the, the, the, you, you, ..."


In [12]:
# sigmoid function
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

In [28]:
class LogisticRegression:
    class Splitter:
        def __init__(self):
            self.train = []
            self.test = []

    class Example:
        """Represents a document with a label. klass is 1 if 'high' and 0 if 'low' by convention.
           words is a string (a single movie review).
        """
        def __init__(self):
            self.klass = -1
            self.words = []

    def __init__(self):
        """
            Logistic Regression initialization - values are already neutralized in the database
        """
        self.category = '' # should be 'valence' or 'arousal'

        self.highWords = {} # words that read high for valence or arousal
        self.lowWords = {} # words that read low for valence or arousal

        # Sorted emotion lexicons
        self.posWords = set()
        self.negWords = set()
        self.angerWords = set()
        self.anticWords = set()
        self.disgustWords = set()
        self.fearWords = set()
        self.joyWords = set()
        self.sadWords = set()
        self.surpriseWords = set()
        self.trustWords = set()

        self.X = [] #input values
        self.Y = [] #true labels
        self.weight = [] #weight vector
        self.b = 0 #bias
        self.pronouns = ['i', 'me', 'mine', 'you', 'your', 'yours']

    def train(self, numIterations):
        for row in train_df.itertuples():
            high = 0
            low = 0
            highCount = 0
            lowCount = 0
            posCount = 0
            negCount = 0
            containsNo = 0
            pronounCount = 0

            # From Emotion Lexicon
            angerCount = 0
            anticCount = 0
            disgustCount = 0
            fearCount = 0
            joyCount = 0
            sadCount = 0
            surpriseCount = 0
            trustCount = 0

            # Changes based on valence, arousal (pulled from VAD lexicon)
            hasHigh1 = 0 # love, abduction
            hasHigh2 = 0 # happy, exorcism
            hasHigh3 = 0 # happily, homicide
            hasLow1 = 0 # toxic, mellow
            hasLow2 = 0 # nightmare, siesta
            hasLow3 = 0 # shit, napping

            for word in row.lyrics:
                if word in self.posWords:
                    posCount += 1
                if word in self.negWords:
                    negCount += 1
                if word in self.angerWords:
                    angerCount += 1
                if word in self.anticWords:
                    anticCount += 1
                if word in self.disgustWords:
                    disgustCount += 1
                if word in self.fearWords:
                    fearCount += 1
                if word in self.joyWords:
                    joyCount += 1
                if word in self.sadWords:
                    sadCount += 1
                if word in self.surpriseWords:
                    surpriseCount += 1
                if word in self.trustWords:
                    trustCount += 1
                if word == 'no':
                    containsNo = 1
                if word in self.pronouns:
                    pronounCount += 1

                if self.category == 'valence':
                    if word in self.highWords:
                        highCount += float(self.highWords[word][0])
                    if word in self.lowWords:
                        lowCount += float(self.lowWords[word][0])
                    if word == 'love':
                        hasHigh1 = 1
                    if word == 'happy':
                        hasHigh2 = 1
                    if word == 'happily':
                        hasHigh3 = 1
                    if word == 'toxic':
                        hasLow1 = 1
                    if word == 'nightmare':
                        hasLow2 = 1
                    if word == 'shit':
                        hasLow3 = 1
                else: # self.category == 'arousal'
                    if word in self.highWords:
                        highCount += float(self.highWords[word][1])
                    if word in self.lowWords:
                        lowCount += float(self.lowWords[word][1])
                    if word == 'abduction':
                        hasHigh1 = 1
                    if word == 'exorcism':
                        hasHigh2 = 1
                    if word == 'homicide':
                        hasHigh3 = 1
                    if word == 'mellow':
                        hasLow1 = 1
                    if word == 'siesta':
                        hasLow2 = 1
                    if word == 'napping':
                        hasLow3 = 1

            if self.category == 'valence':
                klass = row.v_norm
            else:
                klass = row.a_norm
            self.Y = np.append(self.Y, klass)

            high = 1 if klass == 1.0 else 0
            low = 1 if klass == 0 else 0

            wordCount = len(row.lyrics)
            self.X.append([high, low, containsNo, pronounCount, highCount, lowCount, np.log(wordCount), posCount, negCount, angerCount, anticCount, disgustCount, fearCount, joyCount, sadCount, surpriseCount, trustCount, hasHigh1, hasHigh2, hasHigh3, hasLow1, hasLow2, hasLow3]) #missing np.log(wordCount)

            # Initialize self.weight to zeros.
            self.weight = np.zeros((len(self.X[0]), 1))
            lr = 0.001
            self.gradientDescent(lr, numIterations)

    def predict(self, x):
        assert x.shape[0] == 1, "x has the wrong shape. Expected a row vector, got: "+str(x.shape)

        # Classify as high or low
        z = np.dot(x,self.weight) + self.b
        if (sigmoid(z) > 0.5):
            k = 1
        else:
            k = 0
        return k

    def classify(self, example):
        X_test = []

        #feature set
        high = 0
        low = 0
        highCount = 0
        lowCount = 0
        posCount = 0
        negCount = 0
        containsNo = 0
        pronounCount = 0

        # From Emotion Lexicon
        angerCount = 0
        anticCount = 0
        disgustCount = 0
        fearCount = 0
        joyCount = 0
        sadCount = 0
        surpriseCount = 0
        trustCount = 0

        # Changes based on valence, arousal (pulled from VAD lexicon)
        hasHigh1 = 0 # love, abduction
        hasHigh2 = 0 # happy, exorcism
        hasHigh3 = 0 # happily, homicide
        hasLow1 = 0 # toxic, mellow
        hasLow2 = 0 # nightmare, siesta
        hasLow3 = 0 # shit, napping

        for word in example.lyrics:
            if word in self.posWords:
                posCount += 1
            if word in self.negWords:
                negCount += 1
            if word in self.angerWords:
                angerCount += 1
            if word in self.anticWords:
                anticCount += 1
            if word in self.disgustWords:
                disgustCount += 1
            if word in self.fearWords:
                fearCount += 1
            if word in self.joyWords:
                joyCount += 1
            if word in self.sadWords:
                sadCount += 1
            if word in self.surpriseWords:
                surpriseCount += 1
            if word in self.trustWords:
                trustCount += 1
            if word == 'no':
                containsNo = 1
            if word in self.pronouns:
                pronounCount += 1

            if self.category == 'valence':
                if word in self.highWords:
                    highCount += float(self.highWords[word][0])
                if word in self.lowWords:
                    lowCount += float(self.lowWords[word][0])
                if word == 'love':
                    hasHigh1 = 1
                if word == 'happy':
                    hasHigh2 = 1
                if word == 'happily':
                    hasHigh3 = 1
                if word == 'toxic':
                    hasLow1 = 1
                if word == 'nightmare':
                    hasLow2 = 1
                if word == 'shit':
                    hasLow3 = 1
            else: # self.category == 'arousal'
                if word in self.highWords:
                    highCount += float(self.highWords[word][1])
                if word in self.lowWords:
                    lowCount += float(self.lowWords[word][1])
                if word == 'abduction':
                    hasHigh1 = 1
                if word == 'exorcism':
                    hasHigh2 = 1
                if word == 'homicide':
                    hasHigh3 = 1
                if word == 'mellow':
                    hasLow1 = 1
                if word == 'siesta':
                    hasLow2 = 1
                if word == 'napping':
                    hasLow3 = 1

        if self.category == 'valence':
            klass = example.v_norm
        else:
            klass = example.a_norm

        high = 1 if klass == 1.0 else 0
        low = 1 if klass == 0 else 0
        wordCount = len(example.lyrics)

        X_test.append([high, low, containsNo, pronounCount, highCount, lowCount, np.log(wordCount), posCount, negCount, angerCount, anticCount, disgustCount, fearCount, joyCount, sadCount, surpriseCount, trustCount, hasHigh1, hasHigh2, hasHigh3, hasLow1, hasLow2, hasLow3]) # np.log(wordCount)

        return self.predict(np.asarray(X_test))

    def loss(self, a, y):
        return (-1/y.shape[0])*(np.dot(y.T,(np.log(a))) + np.dot((1-y).T,(np.log(1-a))))

    def gradientDescent(self, alpha, numiters):
        self.Y = np.array(self.Y).reshape((-1,1))
        loss = 0
        for i in range(numiters):
            Z = np.dot(self.X, self.weight)+self.b
            A = sigmoid(Z)
            grad = np.dot(np.asarray(self.X).T, (A - self.Y)) / self.Y.shape[0]
            db = np.sum(A - self.Y)/ self.Y.shape[0]
            self.weight -= alpha*grad
            self.b -= alpha*db
            prevLoss = loss
            loss = self.loss(A, self.Y)
            stepSize = abs(prevLoss - loss)

            if stepSize.any() < 0.000001:
                break

            if(i % 25 == 0):
                z = np.dot(self.X, self.weight)+self.b
                a = sigmoid(z)
                #print("loss:" + str(np.squeeze(np.array(self.loss(a, self.Y)))) +"\t %d/%d iterations" % (i, numiters))
                #print(self.weight)

    """
    readFile
    """
    def readFile(self, fileName):
        contents = []
        f = open(fileName, encoding='latin-1')
        contents = f.read()
        f.close()
        return contents

    """
    Starter code for building lexicons.
    Dominance is currently unused
    """
    def buildVADLexicon(self):
        lines = self.readFile('data/NRC-VAD-Lexicon.txt').splitlines()
        for line in lines:
            word,valence,arousal,dominance = line.split('\t')
            if (self.category == 'valence' and float(valence) >= 0.5) or (self.category == 'arousal' and float(arousal) >= 0.5):    # TODO: WHAT'S A GOOD VAL HERE?
                self.highWords[word] = [valence, arousal]
            if (self.category == 'valence' and float(valence) < 0.5) or (self.category == 'arousal' and float(arousal) < 0.5):      # TODO: WHAT"S A GOOD VAL HERE?
                self.lowWords[word] = [valence, arousal]

    def buildEmotionLexicon(self):
        lines = self.readFile('data/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt').splitlines()
        for line in lines:
            word,emotion,value = line.split('\t')
            if emotion == 'positive' and int(value) == 1:
                self.posWords.add(word)
            if emotion == 'negative' and int(value) == 1:
                self.negWords.add(word)
            if emotion == 'anger' and int(value) == 1:
                self.angerWords.add(word)
            if emotion == 'anticipation' and int(value) == 1:
                self.anticWords.add(word)
            if emotion == 'disgust' and int(value) == 1:
                self.disgustWords.add(word)
            if emotion == 'fear' and int(value) == 1:
                self.fearWords.add(word)
            if emotion == 'joy' and int(value) == 1:
                self.joyWords.add(word)
            if emotion == 'sadness' and int(value) == 1:
                self.sadWords.add(word)
            if emotion == 'surprise' and int(value) == 1:
                self.surpriseWords.add(word)
            if emotion == 'trust' and int(value) == 1:
                self.trustWords.add(word)

In [29]:
def evaluate(va, numIters):
    classifier = LogisticRegression()
    classifier.category = va

    classifier.buildVADLexicon()
    classifier.buildEmotionLexicon()

    classifier.train(numIters)

    train_scores = calculate_scores(classifier, train_df)
    test_scores = calculate_scores(classifier, test_df)

    print('Train Accuracy: {}'.format(train_scores[0]))
    print('Test Accuracy: {}'.format(test_scores[0]))

    print('Train Precision: {}'.format(train_scores[1]))
    print('Test Precision: {}'.format(test_scores[1]))

    print('Train Recall: {}'.format(train_scores[2]))
    print('Test Recall: {}'.format(test_scores[2]))

    train_f1 = 2 * (train_scores[1] * train_scores[2]) / (train_scores[1] + train_scores[2])
    test_f1 = 2 * (test_scores[1] * test_scores[2]) / (test_scores[1] + test_scores[2])

    print('Train F1: {}'.format(train_f1))
    print('Test F1: {}'.format(test_f1))

def calculate_scores(classifier, dataset):
    tp = 0.0
    fp = 0.0
    fn = 0.0
    tn = 0.0
    acc = 0
    pre = 0
    rec = 0
    scores = []
    for example in dataset.itertuples():
        guess = classifier.classify(example) # passing a ROW of dataframe
        if (classifier.category == 'valence' and example.v_norm == 1) or (classifier.category == 'arousal' and example.a_norm == 1):
            if (classifier.category == 'valence' and example.v_norm == guess) or (classifier.category == 'arousal' and example.a_norm == guess):
                tp += 1
            else:
                fp += 1
        else:
            if (classifier.category == 'valence' and example.v_norm != guess) or (classifier.category == 'arousal' and example.a_norm != guess):
                fn += 1
            else:
                tn += 1
    acc = (tp + tn) / len(dataset)
    pre = tp / (tp + fp)
    rec = tp / (tp + fn)
    scores.append(acc)
    scores.append(pre)
    scores.append(rec)
    return scores

In [30]:
evaluate('valence', 100)
evaluate('arousal', 100)

Train Accuracy: 0.522
Test Accuracy: 0.49855907780979825
Train Precision: 0.5896103896103896
Test Precision: 0.5961209355390759
Train Recall: 0.4149908592321755
Test Recall: 0.3768481788676524
Train F1: 0.48712446351931327
Test F1: 0.4617764030048608
Train Accuracy: 0.616
Test Accuracy: 0.5857348703170029
Train Precision: 0.7379182156133829
Test Precision: 0.7366591699039051
Train Recall: 0.6203125
Test Recall: 0.5682965299684543
Train F1: 0.6740237691001697
Test F1: 0.6416169530763067
